In [1]:
import os
import pickle
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from metrics import compute_distance
from histograms import compute_descriptor 
import evaluation
import utils

%matplotlib inline

In [ ]:
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
queryset_dir = "{}/masks".format(os.getcwd()) # query set path
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/results".format(os.getcwd()) ## result path

"""
Available descriptors:
    - gray_histogram,
    - hsv_1d_histogram,
    - rgb_1d_histogram
    - ycrcb_1d_histogram,
    - lab_1d_histogram
    - hsv_3d_histogram,
    - rgb_3d_histogram,
    - ycrcb_3d_histogram,
    - lab_3d_histogram,
    - block_histogram
    - spatial_pyramid_histogram
    TODO: adding 2 more descriptors (block + multi-resolution histogram)

Available metrics:
    - cosine,
    - euclidean,
    - hellinger_kernel,
    - chi_square,
    - correlation,
    - intersection
"""
metric = 'hellinger_kernel'
descriptor = 'block_histogram' 
gt_file = '/Users/eternalenvy/Desktop/M1_project/qsd2_w1/gt_corresps_2.pkl'
k = 1

results = []
ground_truth = utils.load_pickle(gt_file)
histograms = utils.load_database(descriptor, dataset_dir, database_dir)

query_set = [fname for fname in sorted(os.listdir(queryset_dir)) if fname.endswith(".png")]
for idx, query in enumerate(query_set):
    print("Query {}: {}, ground-truth: {}".format(idx, query, str(ground_truth[idx][0]).zfill(5)))
    
    img_path = os.path.join(queryset_dir, query)
    
    query_hist = compute_descriptor(descriptor, img_path)
    
    k_nearest = utils.k_nearest_search(histograms, query_hist, ground_truth[idx][0], metric, k)
    
    print("{}-most similar images:".format(k))
    result = []
    for i, image in enumerate(k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        result.append(image[-1])
        if i == k - 1:
            results.append(result)   
    print("==================================")
print(results)    
# utils.save_pickle(result_dir + '/result_' + descriptor + '_top' + str(k) + '.pkl', results)
print(evaluation.mapk(ground_truth, results, k))

In [ ]:
"""
Compute the ratio of correct prediction to see how well it works
"""
count = 0
for i, (a,b) in enumerate(zip(ground_truth, results)):
    if a[0] in b:
        print("OK".format(i))
        count += 1
    else:
        print("Not OK".format(i))
print("{}/30". format(count))

In [ ]:
# Week 2 Task 5 ~~ QST1
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
queryset_dir = "{}/qst1_w2".format(os.getcwd()) # query set path
mask_dir = "{}/qst1_week2/masks".format(os.getcwd()) # masks
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/qst1_week2/result".format(os.getcwd()) ## result path

metric = 'hellinger_kernel'
descriptor = 'block_histogram' 
# gt_file = 'gt_corresps_task5_week2.pkl'
k = 10
# read masks

masks = [os.path.join(mask_dir, f) for f in sorted(os.listdir(mask_dir)) if f.endswith(".png")]
queries = [os.path.join(queryset_dir, f) for f in sorted(os.listdir(queryset_dir)) if f.endswith(".jpg")]

results = []
histograms = utils.load_database(descriptor, dataset_dir, database_dir)
# ground_truth = utils.load_pickle(gt_file)

for idx, (q, m) in enumerate(zip(queries, masks)):
#     print("Query {}: {}, ground-truth: {}".format(idx, q[-9:-4], str(ground_truth[idx][0]).zfill(5)))
    print("Query {}: {}".format(idx, q[-9:-4]))
#     Now for task 5 we have to compute descriptor with masks
    query_hist = []
    if descriptor == 'block_histogram':
        query_hist = block_histogram(q, m)
    else:
        query_hist = spatial_pyramid_histogram(q, m)
    
    k_nearest = utils.k_nearest_search(histograms, query_hist, 0, metric, k)
    
    print("{}-most similar images:".format(k))
    result = []
    temp = []
    for i, image in enumerate(k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        result.append(image[-1])
        if i == k - 1:
            temp.append(result)
            results.append(temp)   
    print("==================================")
print(results)    
# utils.save_pickle(result_dir + '/result.pkl', results)
#print(evaluation.mapk(ground_truth, results, k))

In [2]:
def block_histogram(image_path, mask_path, seg=4):
    """
        compute block histogram of an image in HSV color space
        params:
            path: path pointing to the image to compute
        returns:
            hist: 3D histogram flatted into an 1D array
    """
    image = cv.imread(image_path)
    mask = cv.bitwise_not(cv.imread(mask_path, 0))
    image_tiles = split_image(image, seg)
    mask_tiles = split_image(mask, seg)
    
    hists = []
    for (image_tile, mask_tile) in zip(image_tiles, mask_tiles):  # for each tile
        # rgb
#         rgb = cv.cvtColor(image_tile, cv.COLOR_BGR2RGB)
#         t_hist = cv.calcHist([rgb], [0, 1, 2], mask_tile, [8, 8, 8], [0, 256, 0, 256, 0, 256])
#         t_hist = cv.normalize(t_hist, t_hist).flatten()
#         hists.extend(t_hist)
        
#         # hsv
        hsv = cv.cvtColor(image_tile, cv.COLOR_BGR2HSV)  # convert tile to hsv
        hs = cv.split(hsv)[0:2]  # only calculate for hs
        t_hist = []  # hist for 1 tile
        for c, bin_val, max_val in zip(hs, [180, 64], [180, 256]):
            c_hist = cv.calcHist([c], [0], mask_tile, [bin_val], [0, max_val])  # hist for 1 channel
            c_hist = cv.normalize(c_hist, c_hist).flatten()
            t_hist.extend(c_hist)
        hists.extend(t_hist)
        
    return np.stack(hists).flatten()

def spatial_pyramid_histogram(image_path, mask_path):
    """
        compute spatial pyramid histogram of an image in HSV color space
        params:
            path: path pointing to the image to compute
        returns:
            hist: 3D histogram flatted into an 1D array
    """
    image = cv.imread(image_path)
    mask = cv.bitwise_not(cv.imread(mask_path, 0))
    image_tiles = []
    image_tiles.extend(split_image(image, 1))
    image_tiles.extend(split_image(image, 2))
    image_tiles.extend(split_image(image, 4))
    mask_tiles = []
    mask_tiles.extend(split_image(mask, 1))
    mask_tiles.extend(split_image(mask, 2))
    mask_tiles.extend(split_image(mask, 4))

    hists = []
    for (image_tile, mask_tile) in zip(image_tiles, mask_tiles):  # for each tile
        # rgb
#         rgb = cv.cvtColor(image_tile, cv.COLOR_BGR2RGB)
#         t_hist = cv.calcHist([rgb], [0, 1, 2], mask_tile, [8, 8, 8], [0, 256, 0, 256, 0, 256])
#         t_hist = cv.normalize(t_hist, t_hist).flatten()
#         hists.extend(t_hist)
        
#         # hsv
        hsv = cv.cvtColor(image_tile, cv.COLOR_BGR2HSV)  # convert tile to hsv
        hs = cv.split(hsv)[0:2]  # only calculate for hs
        t_hist = []  # hist for 1 tile
        for c, bin_val, max_val in zip(hs, [180, 64], [180, 256]):
            c_hist = cv.calcHist([c], [0], mask_tile, [bin_val], [0, max_val])  # hist for 1 channel
            c_hist = cv.normalize(c_hist, c_hist).flatten()
            t_hist.extend(c_hist)
        hists.extend(t_hist)
        
    return np.stack(hists).flatten()

def split_image(img, seg=2):
    M = img.shape[0]//seg
    N = img.shape[1]//seg
    tiles = []
    for i in range(0, seg):
        start_x = i*M
        end_x = i*M + M if i != seg - 1 else img.shape[0]
        for j in range(0, seg):
            start_y = j*N
            end_y = j*N + N if j != seg - 1 else img.shape[1]
            tiles.append(img[start_x:end_x, start_y:end_y])
    return tiles

In [3]:
# Week 2 Task 6 ~~ QST2
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
l_queryset_dir = "{}/qst2_week2/L".format(os.getcwd()) # l
r_queryset_dir = "{}/qst2_week2/R".format(os.getcwd()) # r
l_mask_dir = "{}/qst2_week2/L_masks".format(os.getcwd()) # l mask
r_mask_dir = "{}/qst2_week2/R_masks".format(os.getcwd()) # r mask
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/qst2_week2/result/method1".format(os.getcwd()) ## result path

metric = 'hellinger_kernel'
descriptor = 'block_histogram' 
# gt_file = 'gt_corresps_task6_week2.pkl'
k = 10

# read masks

l_masks = [os.path.join(l_mask_dir, f) for f in sorted(os.listdir(l_mask_dir)) if f.endswith(".png")]
r_masks = [os.path.join(r_mask_dir, f) for f in sorted(os.listdir(r_mask_dir)) if f.endswith(".png")]
l_queries = [os.path.join(l_queryset_dir, f) for f in sorted(os.listdir(l_queryset_dir)) if f.endswith(".jpg")]
r_queries = [os.path.join(r_queryset_dir, f) for f in sorted(os.listdir(r_queryset_dir)) if f.endswith(".jpg")]
m_queries = [1, 2, 3, 4, 6, 7, 11, 12, 13, 14, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 29]

results = []
histograms = utils.load_database(descriptor, dataset_dir, database_dir)
# ground_truth = utils.load_pickle(gt_file)

for idx, (lq, rq, lm, rm) in enumerate(zip(l_queries, r_queries, l_masks, r_masks)):
#     print("Query {}: {}, ground-truth: {}".format(idx, lq[-9:-4], str(ground_truth[idx])))
    print("Query {}: {}".format(idx, lq[-9:-4]))
    l_query_hist = []
    if descriptor == 'block_histogram':
        l_query_hist = block_histogram(lq, lm)
    else:
        l_query_hist = spatial_pyramid_histogram(lq, lm)
    if idx not in m_queries:
        r_query_hist = []
        if descriptor == 'block_histogram':
            r_query_hist = block_histogram(rq, rm)
        else:
            r_query_hist = spatial_pyramid_histogram(rq, rm)
        
    l_k_nearest = utils.k_nearest_search(histograms, l_query_hist, 0, metric, k)
    if idx not in m_queries:
        r_k_nearest = utils.k_nearest_search(histograms, r_query_hist, 0, metric, k)
    
    result = []
    print("{}-most similar images:".format(k))
    l_result = []
    for i, image in enumerate(l_k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        l_result.append(image[-1])
    result.append(l_result)
    if idx not in m_queries:
        r_result = []
        for i, image in enumerate(r_k_nearest):
            print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
            r_result.append(image[-1])
        result.append(r_result)
    results.append(result)
    print("==================================")
print(results)    
utils.save_pickle(result_dir + '/result.pkl', results)

Query 0: 00000
10-most similar images:
1. 00212.jpg, score = 0.481773592971
2. 00270.jpg, score = 0.501632630837
3. 00146.jpg, score = 0.515810000253
4. 00035.jpg, score = 0.522161580635
5. 00217.jpg, score = 0.529556608681
6. 00128.jpg, score = 0.534976263382
7. 00214.jpg, score = 0.5353511841
8. 00162.jpg, score = 0.538504823902
9. 00085.jpg, score = 0.538885910321
10. 00046.jpg, score = 0.545564907786
1. 00027.jpg, score = 0.330890487734
2. 00110.jpg, score = 0.400668608801
3. 00154.jpg, score = 0.436287947537
4. 00031.jpg, score = 0.438511664816
5. 00105.jpg, score = 0.452167108111
6. 00037.jpg, score = 0.454101409412
7. 00156.jpg, score = 0.458206845361
8. 00029.jpg, score = 0.46083137888
9. 00259.jpg, score = 0.461459581842
10. 00276.jpg, score = 0.468237819712
Query 1: 00001
10-most similar images:
1. 00130.jpg, score = 0.325965477442
2. 00271.jpg, score = 0.379845362215
3. 00225.jpg, score = 0.385036013574
4. 00057.jpg, score = 0.396124170054
5. 00244.jpg, score = 0.39950384409

10-most similar images:
1. 00132.jpg, score = 0.344890573232
2. 00225.jpg, score = 0.409239796531
3. 00271.jpg, score = 0.420566636377
4. 00130.jpg, score = 0.424049134087
5. 00133.jpg, score = 0.426425215808
6. 00058.jpg, score = 0.43313183525
7. 00103.jpg, score = 0.434365567674
8. 00057.jpg, score = 0.438194955576
9. 00081.jpg, score = 0.439319137806
10. 00244.jpg, score = 0.441686687943
1. 00225.jpg, score = 0.381659650792
2. 00037.jpg, score = 0.393286598368
3. 00271.jpg, score = 0.39546282452
4. 00130.jpg, score = 0.407646784995
5. 00072.jpg, score = 0.411367674722
6. 00081.jpg, score = 0.420026894896
7. 00154.jpg, score = 0.425168781575
8. 00103.jpg, score = 0.430836055401
9. 00057.jpg, score = 0.432585704611
10. 00163.jpg, score = 0.434287623978
Query 16: 00016
10-most similar images:
1. 00163.jpg, score = 0.343874978026
2. 00137.jpg, score = 0.386861390594
3. 00016.jpg, score = 0.389320801774
4. 00135.jpg, score = 0.395259176571
5. 00051.jpg, score = 0.397388785644
6. 00103.jp

In [ ]:
gt_extracted = []
for l in ground_truth:
    for e in l:
        gt_extracted.append([e])
results_extracted = []
for l in results:
    for e in l:
        results_extracted.append(e)
print(evaluation.mapk(gt_extracted, results_extracted, k))